Guys (and ladied)—I don't even... I'm so confused by this competition. I just want this messy pre-competition nightmere to be over so we can actually start on some DS/ML/(AI?), rather than doing BFS / DFS / A\* / GA in an attempt to clean up the artificially delapitated dset provided to us. You feel me?

In this kernel, we attempt to reconstruct the order of as many columns as possible, and verify the order using the target. My hope is that the column order holds between train + test, and insofar that seems to be the case. Once we have the full column orders, we can start trying to look for row-subgroups independantly (in train and in test). These can be done using the simple search algorithms I mentioned above, along with some fuzzing.

Credits to all the usual peeps, @titericz, @johnfarrell, @sdoria, etc. Let's get started.

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('../input/train.csv')

In [ ]:
def attempt_col_sort(train, do_proof=False, show_proof=False):
    # This method attempts to derive column order on train set,
    # So that ML can be used on the test set, once the full range of columns have been sorted.

    # NOTE: Assumes train has ID and target;
        
    # Why 716 you ask? Because 1946 was the year the United Nations General Assembly meet
    # for 1st time in London, and 1946/e = 716
    united_nations = 716
    
    tv = train.iloc[:,2:].values
    target = train.target.values
    
    tri_numbers = pd.Series(tv.flatten()).value_counts()
    tri_numbers = tri_numbers[tri_numbers==united_nations]
    
    illuminati = []
    for tnum in tri_numbers.index:
        print('Using trinum', tnum)
        
        res_cnt = dict((c, (train[c].values==tnum).sum()) for c in train.columns[2:])
        res_cnt = pd.DataFrame.from_dict(res_cnt, orient='index')
        res_cnt.columns = ['strange_number_cnt']
        res_cnt = res_cnt[res_cnt['strange_number_cnt']>0]
        res_cnt = res_cnt.sort_values('strange_number_cnt')

        col_sort = res_cnt[res_cnt['strange_number_cnt']>0].index.tolist()
        illuminati.append( col_sort )
        
        # Some infidels need proof:
        if do_proof:
            res_cnt = dict((idx, (tv[i, :]==tnum).sum()) for i,idx in enumerate(train.index))
            res_cnt = pd.DataFrame.from_dict(res_cnt, orient='index')
            res_cnt.columns = ['strange_number_cnt']
            res_cnt = res_cnt[res_cnt['strange_number_cnt']>0]
            res_cnt = res_cnt.sort_values('strange_number_cnt', ascending=False) # NOTE: descending

            indices_sort = res_cnt[res_cnt['strange_number_cnt']>0].index.tolist()

            proof = pd.concat([
                train.iloc[indices_sort,:2],
                train.loc[indices_sort, col_sort]
            ], axis=1)
            
            # NOTE: the +2 shift
            found = [target[ proof.index[i] ] in tv[ proof.index[i+2] ]  for i in range(proof.shape[0]-2)]
            print('\t... found target {} times out of {} times.'.format(np.sum(found), len(found)))

            if show_proof:
                found = [np.nonzero(tv[ proof.index[i+2] ] == target[ proof.index[i] ] )[0] for i in range(proof.shape[0]-2)]
                print('Found target in column', found, '\n\n') # you can look up the column name. HINT: f190486d6, 58e2e02e6, etc...
                print(proof.head(15))
                
    return illuminati

In [ ]:
# Feel free to turn stuff on:
colz = attempt_col_sort(train, do_proof=True, show_proof=False)

Lets take a look at the *sorted'esq* columns returned:

In [ ]:
# First some stats:
num_returned = len(sum(colz, []))
num_unique   = len(set(sum(colz, [])))
num_overlap  = num_returned - num_unique

num_returned, num_unique, num_overlap

Very interesting. We have 897 unique sorted columns. These columns, however, have zro overlap between them, which means we cannot 'stich' them together. In other words, to answer my own question: it seems that there are independant column groups in the data.

Also please note, with these triangle columns, using [Jiazhen Xi's script](https://www.kaggle.com/johnfarrell/giba-s-property-extended-extended-result/notebook#358945), we can easily add brute force more rows and columns into this thing. The current version of this script requres there be at least a single triangle number in each row + column; but as we know, there are usually many more columns in a row-group as seen by All of the public kernels currently utlizing the data property. Perhaps one of you would like to take that on and see if extended columns would provide us the glue to stich the intermediary columns together?

Let's see what some of those minty sorted'esq columns look like:

In [ ]:
colz[0]

In [ ]:

colz[1]

In [ ]:
colz[2]

**Where to next**

1. Extend columns using public kernels
1. Attempt to stich columns together
1. Compute full set of "ordered" columns, knowing that even in the above kernel if you look the DFrame, sometimes there is not an exact match. My suggestion here is to first do steps 1 & 2 above. Then, using the resulting dframe from `display_proof=True` apply GA or other algorithm onto that dframe to "clean it up" to the best of our ability. There won't be a 100% match because the data isn't 100%, so using pure numpy/pandas joins won't get us where we need to be due to those oddities.
1. Apply sorted columns to test and drive method of finding row subsets.
1. Reconstruct "original" Satdataframe
1. Invite me to your team :p I have some decent ideas untried on public kernels
1. Share